In [3]:
from utilities.logger import Logger
from datetime import datetime
import os 
import pandas as pd
from typing import List, Dict, Union


run_id = datetime.now().strftime('%Y%m%d_%H%M')
run_id

logger = Logger(run_id)
logger.initialization()

In [9]:
base_path = os.path.join(os.getcwd(), "prd-trd", "2_referentiel")
os.makedirs(base_path, exist_ok=True)
logger.log_info(f"base path is: {base_path}")

base path is: c:\Users\beAtz\OneDrive\Bureau\WTA_PROJECT\prd-trd\2_referentiel


# REFERENTIEL JOUEURS

In [5]:

def create_player_referential(df: pd.DataFrame) -> pd.DataFrame:
    """
    Takes a dataframe with columns related to two players' information and
    returns a new dataframe with one player's information per row.
    
    Parameters:
    - df (pd.DataFrame): Input dataframe with columns for two players.

    Returns:
    - pd.DataFrame: Reformatted dataframe with one player's info per row.
    """
    
    # Extract player 1 and player 2 data, and rename columns
    df_player_1 = df[['player_1_name', 'Date of Birth_player_1', 'Height_player_1', 'Plays_player_1']]
    df_player_1.columns = ['player_name', 'date_of_birth', 'height_player', 'plays_player']

    df_player_2 = df[['player_2_name', 'Date of Birth_player_2', 'Height_player_2', 'Plays_player_2']]
    df_player_2.columns = ['player_name', 'date_of_birth', 'height_player', 'plays_player']

    # Concatenate the dataframes
    player_referential = pd.concat([df_player_1, df_player_2], ignore_index=True)
    player_referential.drop_duplicates(subset='player_name', inplace=True)


    return player_referential


In [7]:
def create_player_referential_over_data() -> pd.DataFrame:
    """
    Iterates through the files for each year and each tournament, 
    merging 'matches' and 'matches_details' data.
    Then it creates a player referential based on the merged data.
    
    Returns:
        pd.DataFrame: A DataFrame containing the player referential.
    """
    # Your existing logger method, uncomment it if you are actually using it
    # logger.log_info(f"correcting data for years {','.join(years)}")

    base_path = os.path.join(os.getcwd(), "prd-trd", "0_raw")
    years = os.listdir(base_path)

    # Create an empty DataFrame to store concatenated results
    concatenated_data = pd.DataFrame()

    # Iterating over the years
    for year in years:
        year_path = os.path.join(base_path, year)

        # Iterating over the tournaments of this year
        for tourney in os.listdir(year_path):
            tourney_path = os.path.join(year_path, tourney)
            fichiers_dossier = os.listdir(tourney_path)
            
            # Convert fichiers_dossier to a set for efficient membership testing
            fichiers_set = set(fichiers_dossier)

            if {"matches.csv", "matches_details.csv"}.issubset(fichiers_set):
                matches = pd.read_csv(os.path.join(tourney_path, "matches.csv"), sep=";")
                matches_lit = matches[["player_1_name", "player_2_name", "url_match_stats"]]
                matches_details = pd.read_csv(os.path.join(tourney_path, "matches_details.csv"), sep=";")
                cols_details = [col for elt in range(1, 3) for col in [f"Date of Birth_player_{elt}", f"Height_player_{elt}", f"Plays_player_{elt}"]] + ["url_match_stats"]
                matches_details_lit = matches_details[cols_details]
                merged_matches_info = pd.merge(matches_lit, matches_details_lit, on="url_match_stats", how="left")
                
                # Append the merged data to the concatenated_data DataFrame
                concatenated_data = pd.concat([concatenated_data, merged_matches_info], ignore_index=True)

    # Assuming 'create_player_referential' is a function you've defined elsewhere
    players_referential = create_player_referential(concatenated_data)
    
    return players_referential


In [10]:
referential_players = create_player_referential_over_data()
referential_players.to_csv(os.path.join(base_path, "referentiel_joueurs.csv"), sep=";", index=False)

# TOURNAMENT REFERENTIAL

In [11]:
from typing import List, Union
import os
import pandas as pd

def create_tourney_referential_over_data() -> pd.DataFrame:
    """
    Iterates through the files for each year and each tournament, 
    merging 'matches' and 'matches_details' data.
    Then it creates a player referential based on the merged data.
    
    Returns:
        pd.DataFrame: A DataFrame containing the player referential.
    """
    # Your existing logger method, uncomment it if you are actually using it
    # logger.log_info(f"correcting data for years {','.join(years)}")

    base_path = os.path.join(os.getcwd(), "prd-trd", "0_raw")
    years = os.listdir(base_path)

    # Create an empty DataFrame to store concatenated results
    concatenated_data = pd.DataFrame()

    # Iterating over the years
    for year in years:
        year_path = os.path.join(base_path, year)

        # Iterating over the tournaments of this year
        for tourney in os.listdir(year_path):
            tourney_path = os.path.join(year_path, tourney)
            
            # Convert fichiers_dossier to a set for efficient membership testing
            tournament_name = tourney_path.split("\\")[-1]
            tournament_path = os.path.join(tourney_path, f"{tournament_name}.csv")
            data = pd.read_csv(tournament_path, sep=";")
            to_select = ["title", "location", "surface"]
            data_lit = data[to_select]
                
            # Append the merged data to the concatenated_data DataFrame
            concatenated_data = pd.concat([concatenated_data, data_lit], ignore_index=True)

    
    return concatenated_data


In [12]:
tourneys = create_tourney_referential_over_data()
tourneys.to_csv(os.path.join(base_path, "referentiel_tournois.csv"), sep=";", index=False)